In [1]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
import datetime
from time import sleep
from bs4 import BeautifulSoup
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np

In [2]:
def SearchElement(element, tag, soup, soloElement = True, rawElement=False):

    returnArray = []

    try:
        results = soup.find_all(element, tag)

        if rawElement:
            return results

        for result in results:
            returnArray.append(result.text.strip())

        if soloElement:
            return (returnArray[0])
        else:
            return(returnArray)
    except BaseException:
        return("")

# Link Crawl section

In [4]:
url = 'https://www.livebitcoinnews.com/news/bitcoin-news/page/'
result = []
stop = False
for i in range(1,100):
    newurl = url+""+str(i)
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=newurl, headers=headers) 
    page = urlopen(req)
    
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    hrefs = SearchElement("h3", "title", soup, soloElement=False, rawElement=True);
    times = SearchElement("span", "time", soup, soloElement=False, rawElement = False);

    for href in hrefs:
        res = href.find('a')['href']
        if res == 'https://blockchain-life.com/europe/':
            continue
        result.append(res)
        
    for time in times:
        print(time)
        if "February" in time and "2019" in time:
            print("I am stopping I found critical date")          
            stop = True
            break
    if stop:
        break
    sleep(0.3)

urlsDf = pd.DataFrame(result, columns =['urls'])


September 26, 2019 · 1:00 pm
September 25, 2019 · 3:00 pm
September 25, 2019 · 1:00 pm
September 25, 2019 · 4:08 am
September 24, 2019 · 5:00 pm
September 24, 2019 · 3:00 pm
September 24, 2019 · 1:00 pm
September 23, 2019 · 3:00 pm
September 23, 2019 · 1:00 pm
September 22, 2019 · 4:30 pm
September 22, 2019 · 2:30 pm
September 21, 2019 · 5:00 pm
September 20, 2019 · 3:00 pm
September 20, 2019 · 1:00 pm
September 19, 2019 · 3:00 pm
September 19, 2019 · 1:00 pm
September 18, 2019 · 3:00 pm
September 18, 2019 · 4:08 am
September 17, 2019 · 5:00 pm
September 17, 2019 · 3:00 pm
September 17, 2019 · 1:00 pm
September 16, 2019 · 3:00 pm
September 16, 2019 · 1:00 pm
September 15, 2019 · 6:30 pm
September 15, 2019 · 6:00 pm
September 15, 2019 · 5:30 pm
September 13, 2019 · 3:00 pm
September 13, 2019 · 1:00 pm
September 12, 2019 · 5:00 pm
September 12, 2019 · 3:00 pm
September 11, 2019 · 5:00 pm
September 11, 2019 · 1:00 pm
September 11, 2019 · 3:08 am
September 10, 2019 · 5:00 pm
September 10, 

In [5]:
urlsDf.to_csv("livebitcoinlinks.csv", encoding='utf-8', index=False)

# Content Crawl

#### Work with soup
soup.find_all('a', class_='featured-image')  
href['href'] - where href is result of find('a')  
soup.find(id="link3")
_tag.text - jadro tagu  
_tag.next_sibling - text dalsieho tagu

In [7]:
def CheckValidUrl(url):
    if "https://" not in url:
            print("I am skipping wrong html")
            return False
    return True
        
def GetUrlSoup(url):
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=url, headers=headers) 
    page = urlopen(req)
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    return soup;

def GetAuthorOfPublication(soup):
    author = soup.find('span', class_="author")
    author = author.find('a').text 
    return author;

def GetDateOfPublish(soup):
    date = soup.find('span', class_="date").text
    date_time = datetime.datetime.strptime(date, '%B %d, %Y')
    return date_time

def GetTimeOfPublish(soup):
    date = soup.find('span', class_="date").text
    time = soup.find('span', class_="time").text.replace("·", "").replace(" ", "")
    datetimeText = date+" "+time
    date_time = datetime.datetime.strptime(datetimeText, '%B %d, %Y %I:%M%p')
    return date_time

def GetHeader(soup):
    header = soup.find('h1', class_="title")
    header = header.find('a').text
    return header.encode('ascii', 'ignore').decode("utf-8")

def GetSubHeaders(soup):
    subHeaders = []
    divContent = soup.find('div', class_="post-content")
    headers = divContent.find_all('h2')
    for header in headers:
        subHeaders.append((header.text).encode('ascii', 'ignore').decode("utf-8"))
    return subHeaders
    
def GetContent(soup):
    TextContent = ""
    divContent = soup.find('div', class_="post-content")
    textElements = divContent.find_all('p')
    for p in textElements:
        if "You must be" in p.text:
            continue
        TextContent = TextContent +" "+ p.text
    return TextContent.encode('ascii', 'ignore').decode("utf-8")
    
def GetTags(soup):
    tags=[]
    sectionTags = soup.find('span', class_="tags")
    stream = sectionTags.find_all('a')
    for tag in stream:
        tags.append((tag.text).encode('ascii', 'ignore').decode("utf-8"))
    return tags
    

In [8]:
resultPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
with open('livebitcoinlinks.csv') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        rowPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
        crawlurl=row[0]
        if not CheckValidUrl(crawlurl):
            continue;
        soup = GetUrlSoup(crawlurl)
        
        #content Crawl
        Source = "www.livebitcoinnews.com";
        Url = crawlurl;
        Author = GetAuthorOfPublication(soup);
        Date = GetDateOfPublish(soup);
        DateTime = GetTimeOfPublish(soup);
        HeaderText = GetHeader(soup);
        SubHeadersText = GetSubHeaders(soup);
        ContentText = GetContent(soup);
        Tags = GetTags(soup);
    
    
        rowPd.at[0,'source'] = Source
        rowPd.at[0,'url'] = Url
        rowPd.at[0,'author'] = Author
        rowPd.at[0,'date'] = Date
        rowPd.at[0,'datetime'] = DateTime
        rowPd.at[0,'headertext'] = HeaderText
        rowPd.at[0,'subheaders'] = SubHeadersText
        rowPd.at[0,'contenttext'] = ContentText
        rowPd.at[0,'tags'] = Tags
        print(Date)
        frames = [resultPd, rowPd]
        resultPd = pd.concat(frames)


I am skipping wrong html
2019-09-26 00:00:00
2019-09-25 00:00:00
2019-09-25 00:00:00
2019-09-25 00:00:00
2019-09-24 00:00:00
2019-09-24 00:00:00
2019-09-24 00:00:00
2019-09-23 00:00:00
2019-09-23 00:00:00
2019-09-22 00:00:00
2019-09-22 00:00:00
2019-09-21 00:00:00
2019-09-20 00:00:00
2019-09-20 00:00:00
2019-09-19 00:00:00
2019-09-19 00:00:00
2019-09-18 00:00:00
2019-09-18 00:00:00
2019-09-17 00:00:00
2019-09-17 00:00:00
2019-09-17 00:00:00
2019-09-16 00:00:00
2019-09-16 00:00:00
2019-09-15 00:00:00
2019-09-15 00:00:00
2019-09-15 00:00:00
2019-09-13 00:00:00
2019-09-13 00:00:00
2019-09-12 00:00:00
2019-09-12 00:00:00
2019-09-11 00:00:00
2019-09-11 00:00:00
2019-09-11 00:00:00
2019-09-10 00:00:00
2019-09-10 00:00:00
2019-09-10 00:00:00
2019-09-09 00:00:00
2019-09-09 00:00:00
2019-09-08 00:00:00
2019-09-08 00:00:00
2019-09-07 00:00:00
2019-09-07 00:00:00
2019-09-06 00:00:00
2019-09-05 00:00:00
2019-09-05 00:00:00
2019-09-05 00:00:00
2019-09-04 00:00:00
2019-09-04 00:00:00
2019-09-04 00:0

In [9]:
resultPd.head()


,source,url,author,date,datetime,headertext,subheaders,contenttext,tags
0,www.livebitcoinnews.com,https://www.livebitcoinnews.com/is-bitcoin-pot...,Nick Marinoff,2019-09-26 00:00:00,2019-09-26 13:00:00,Is Bitcoin Potentially Being Manipulated Again?,"[Bitcoin: the Target of More Tricks?, Is Bakkt...","Is bitcoin being manipulated again? In 2018, ...","[Bakkt, bitcoin, Bitcoin Futures, cme]"
0,www.livebitcoinnews.com,https://www.livebitcoinnews.com/how-wall-stree...,Nick Marinoff,2019-09-25 00:00:00,2019-09-25 15:00:00,How Wall Street Could Be Bitcoins Biggest Fina...,"[What Wall Street Is Doing for BTC, A Rough Ro...","With bitcoins recent fall into the $8,000 ran...","[Bakkt, bitcoin, Wall Street]"
0,www.livebitcoinnews.com,https://www.livebitcoinnews.com/what-could-hav...,Nick Marinoff,2019-09-25 00:00:00,2019-09-25 13:00:00,What Could Have Made Bitcoin Sink So Dramatica...,"[Bitcoin and Crypto Falls Heavily, What Else C...","Well, after yesterdays crypto bloodbath, prob...","[Bakkt, bitcoin, BitMEX, crypto]"
0,www.livebitcoinnews.com,https://www.livebitcoinnews.com/bitcoin-price-...,Aayush Jindal,2019-09-25 00:00:00,2019-09-25 04:08:00,Bitcoin Price Analysis: BTC Tumbles Below $9K ...,"[Bitcoin Price Analysis, Technical indicators]",Bitcoin price is back in a downtrend below $9...,"[bitcoin, BTC]"
0,www.livebitcoinnews.com,https://www.livebitcoinnews.com/bitcoin-atms-a...,Nick Marinoff,2019-09-24 00:00:00,2019-09-24 17:00:00,Bitcoin ATMs Are Popping Up At an Alarming Rate,"[Bitcoin ATMs Are Expanding Quickly, Making Cr...",It seems likecryptocurrency and bitcoin ATMs ...,"[Bitcoin ATMs, Bitcoin Depot, Latin America, u..."


In [10]:
resultPd.to_csv("livebitcoinnews_data.csv", encoding='utf-8', index=False)
